In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import shutil

In [ ]:
import time

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

<ipython-input-6-8945a9c4fdc8>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [ ]:
db_path = '/content/drive/MyDrive/isa_proj/stock_data.db'
output_path = '/content/drive/MyDrive/isa_proj/output_derived.db'

In [ ]:
conn = sqlite3.connect(db_path)

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("select industry, count(*) from stock_prices group by industry order by count(*) desc")
res = cursor.fetchall()

In [ ]:
selected_industries = ['Banks—Regional', 'Software—Application', 'Software—Infrastructure', 'Information Technology Services',
                       'Capital Markets', 'Internet Content & Information', 'Electronic Components', 'Consumer Electronics',
                       'Entertainment', 'Medical Devices']

In [ ]:
res

[('Banks—Regional', 291573),
 ('Biotechnology', 184340),
 ('Asset Management', 162198),
 ('Software—Application', 103759),
 ('Software—Infrastructure', 64976),
 ('Specialty Industrial Machinery', 59372),
 ('Medical Devices', 56784),
 ('Drug Manufacturers—Specialty & Generic', 54563),
 ('Oil & Gas E&P', 54396),
 ('Specialty Chemicals', 52495),
 ('Telecom Services', 52039),
 ('Packaged Foods', 49433),
 ('Communication Equipment', 47435),
 ('Semiconductors', 40161),
 ('Shell Companies', 39653),
 ('Aerospace & Defense', 39503),
 ('Auto Parts', 38522),
 ('Information Technology Services', 37295),
 ('Internet Content & Information', 36774),
 ('Capital Markets', 34605),
 ('Insurance—Property & Casualty', 32762),
 ('Restaurants', 31594),
 ('Utilities—Regulated Electric', 30873),
 ('Credit Services', 30671),
 ('Diagnostics & Research', 30657),
 ('Oil & Gas Equipment & Services', 30542),
 ('Engineering & Construction', 29874),
 ('Electronic Components', 28890),
 ('Medical Instruments & Supplies'

In [ ]:
cursor.execute("select * from stock_prices limit 1")
res = cursor.fetchall()
res

[(1,
  '2021-01-04',
  14.83269216966404,
  14.997296931078534,
  14.476049682740546,
  14.649799346923828,
  1638500,
  0.0,
  0.0,
  'TS',
  'Tenaris S.A.',
  'oil-gas-equipment-services',
  'USD',
  'Energy',
  'argentina')]

In [ ]:
df_tech = pd.read_sql("select open, high, low, close, volume, dividends, stock_splits, stock from stock_prices where industry='Consumer Electronics'", conn)

In [ ]:
len(df_tech)

3012

In [ ]:
df_financial = pd.read_sql("select open, high, low, close, volume, dividends, stock_splits, stock from stock_prices where sector='Financial Services'", conn)

In [ ]:
df_healthcare = pd.read_sql("select open, high, low, close, volume, dividends, stock_splits, stock from stock_prices where sector='Healthcare'", conn)

In [ ]:
def preprocessing(df_sector_unique, min_max_needed=False):
  encoder = OneHotEncoder(sparse=False)
  ohe_cols = encoder.fit_transform(df_sector_unique[['Stock']])
  ohe_col_names = encoder.get_feature_names_out(['Stock'])
  df_ohe = pd.DataFrame(ohe_cols, columns=ohe_col_names, index=df_sector_unique.index)

    # Combine the one-hot encoded columns with the original dataframe
  df_sector_encoded = pd.concat([df_sector_unique, df_ohe], axis=1)

    # Perform normalization on non-OHE columns
  non_ohe_cols = [col for col in df_sector_encoded.columns if col not in ['Stock'] + list(ohe_col_names)]
  scaler = StandardScaler()
  if min_max_needed:
    scaler = MinMaxScaler()
  df_sector_encoded[non_ohe_cols] = scaler.fit_transform(df_sector_encoded[non_ohe_cols])

  return df_sector_encoded, ohe_col_names

In [ ]:
def perform_clustering(df_sector_unique):
    df_sector_encoded, ohe_col_names = preprocessing(df_sector_unique, True)

    # Perform clustering
    kmeans = MiniBatchKMeans(n_clusters=3, random_state=42)
    df_clustered = df_sector_encoded.copy()  # Create a copy to avoid modifying the original DataFrame
    df_clustered['Cluster'] = kmeans.fit_predict(df_sector_encoded.drop(columns=['Stock']))

    #df_sector_unique['Cluster'] = df_clustered['Cluster']


    df_clustered = df_clustered.drop(columns=list(ohe_col_names))


    return df_clustered

In [ ]:
qry = "select count(*) from stock_prices where stock='AAPL'"
cursor.execute(qry)
res = cursor.fetchall()
print(res)

[(753,)]


In [ ]:
df_tech.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits',
       'Stock'],
      dtype='object')

In [ ]:
df_tech_clustered = perform_clustering(df_tech)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
df_tech_clustered

,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Stock,Cluster
0,0.883053,0.883053,0.888977,0.888977,5.116851e-07,0.0,0.0,CSIOY,1
1,0.876191,0.876191,0.882069,0.882069,5.116851e-07,0.0,0.0,CSIOY,1
2,0.868786,0.868786,0.874615,0.874615,1.535055e-06,0.0,0.0,CSIOY,1
3,0.868786,0.868786,0.874615,0.874615,5.116851e-07,0.0,0.0,CSIOY,1
4,0.872883,0.872883,0.852202,0.854736,5.116851e-07,0.0,0.0,CSIOY,1
...,...,...,...,...,...,...,...,...,...
10537,0.000000,0.000000,0.000000,0.000000,3.581796e-06,0.0,0.0,ANDR,2
10538,0.000000,0.000000,0.000000,0.000000,4.195818e-05,0.0,0.0,ANDR,2
10539,0.000000,0.000000,0.000000,0.000000,1.150268e-03,0.0,0.0,ANDR,2
10540,0.000000,0.000000,0.000000,0.000000,5.116851e-06,0.0,0.0,ANDR,2


In [ ]:
cols = list(df_tech.columns)
cols = list(set(cols) - set(['Cluster', 'Stock']))

In [ ]:
cols = ['Close', 'Low', 'High', 'Open', 'Volume', 'Dividends', 'Price_Range', 'Pct_Change', 'SMA_7',
 'EMA_7', 'Volatility', 'RSI', 'MACD', 'MACD_Signal', 'VWAP', 'RiskAdjustedReturn',
 'IndustryRankRSI', 'CloseToVWAPRatio', 'MACDSignalDiff', 'Percentage_Bandwidth']

In [ ]:
def feature_summary_statistics(df_clustered, cluster_col, cols):
    # Group DataFrame by cluster label
    grouped = df_clustered.groupby(cluster_col)

    # Initialize an empty DataFrame to store the summary statistics
    summary_stats_df = pd.DataFrame(columns=['Feature', 'Cluster', 'Mean', 'Median', 'Std', 'Min', 'Max'])

    # Calculate summary statistics for each feature within each cluster
    for col in cols:
        for cluster_label, group_data in grouped:
            mean = group_data[col].mean()
            median = group_data[col].median()
            std = group_data[col].std()
            min_val = group_data[col].min()
            max_val = group_data[col].max()

            # Append the summary statistics to the DataFrame
            summary_stats_df = summary_stats_df.append({'Feature': col, 'Cluster': cluster_label,
                                                         'Mean': mean, 'Median': median,
                                                         'Std': std, 'Min': min_val, 'Max': max_val},
                                                        ignore_index=True)

    return summary_stats_df

# Call the function to get summary statistics for each feature
# feature_summary = feature_summary_statistics(df_tech_clustered, 'Cluster', cols)

# # Display the summary statistics for each feature
# print(feature_summary)


In [ ]:
def get_cluster_risk_mapping(feature_summary):
  features_list = ['Close', 'Low', 'High', 'Open', 'Volume', 'Dividends', 'Price_Range', 'Pct_Change', 'SMA_7',
 'EMA_7', 'Volatility', 'RSI', 'MACD', 'MACD_Signal', 'VWAP', 'RiskAdjustedReturn',
 'IndustryRankRSI', 'CloseToVWAPRatio', 'MACDSignalDiff', 'Percentage_Bandwidth']
  std_devs = feature_summary[feature_summary['Feature'].isin(features_list)] \
            .groupby('Cluster')['Std'].mean().reset_index()

  # Sort clusters by their mean standard deviation
  std_devs = std_devs.sort_values(by='Std')

  # Assign risk levels based on sorted standard deviations
  risk_levels = ['Low Risk', 'Medium Risk', 'High Risk']
  std_devs['Risk_Level'] = risk_levels[:len(std_devs)]

  cluster_risk_mapping = {}
  for idx in range(len(std_devs)):
    cluster_risk_mapping[std_devs.iloc[idx]['Cluster']] = std_devs.iloc[idx]['Risk_Level']

  return cluster_risk_mapping


In [ ]:
class AutoencoderHyperModel(HyperModel):
    def __init__(self, input_dim):
        self.input_dim = input_dim

    def build(self, hp):
        encoding_dim = hp.Choice('encoding_dim', [16, 32, 64, 128])

        input_layer = Input(shape=(self.input_dim,))
        encoded = Dense(encoding_dim, activation='relu')(input_layer)
        decoded = Dense(self.input_dim, activation='sigmoid')(encoded)

        autoencoder = Model(input_layer, decoded)

        # You can also tune the learning rate and optimizer
        lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        optimizer_name = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

        if optimizer_name == 'adam':
            optimizer = Adam(learning_rate=lr)
        elif optimizer_name == 'sgd':
            optimizer = SGD(learning_rate=lr)
        elif optimizer_name == 'rmsprop':
            optimizer = RMSprop(learning_rate=lr)

        autoencoder.compile(optimizer=optimizer, loss='mse')

        return autoencoder

In [ ]:
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
def calculate_macd(data, slow=26, fast=12):
    ema_fast = data.ewm(span=fast, adjust=False).mean()
    ema_slow = data.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

In [ ]:
def calculate_bollinger_bands(data, window=20):
    sma = data.rolling(window=window).mean()
    std = data.rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, sma, lower_band

In [ ]:
def perform_autoencoder_clustering(df_tech):

  df_tech['Price_Range'] = df_tech['High'] - df_tech['Low']

  # Calculate daily percentage change
  df_tech['Pct_Change'] = df_tech['Close'].pct_change() * 100

  # Calculate a simple moving average (SMA) over a 7-day window
  df_tech['SMA_7'] = df_tech['Close'].rolling(window=7).mean()

  # Calculate exponential moving average (EMA) for a faster response to recent price changes, over a 7-day window
  df_tech['EMA_7'] = df_tech['Close'].ewm(span=7, adjust=False).mean()

  # Calculate volatility (standard deviation of daily price change)
  df_tech['Volatility'] = df_tech['Pct_Change'].rolling(window=7).std()

  df_tech.dropna(inplace=True)

  df_tech['RSI'] = calculate_rsi(df_tech['Close'])

  df_tech['MACD'], df_tech['MACD_Signal'] = calculate_macd(df_tech['Close'])

  df_tech['BB_Upper'], df_tech['BB_Middle'], df_tech['BB_Lower'] = calculate_bollinger_bands(df_tech['Close'])

  df_tech['VWAP'] = (df_tech['Volume'] * (df_tech['High'] + df_tech['Low'] + df_tech['Close']) / 3).cumsum() / df_tech['Volume'].cumsum()

  df_tech['RiskAdjustedReturn'] = df_tech['Pct_Change'] / df_tech['Volatility']

  df_tech['IndustryRankRSI'] = df_tech['RSI'].rank(pct=True)

  df_tech['CloseToVWAPRatio'] = df_tech['Close'] / df_tech['VWAP']

  df_tech['MACDSignalDiff'] = df_tech['MACD'] - df_tech['MACD_Signal']

  df_tech['Percentage_Bandwidth'] = (df_tech['BB_Upper'] - df_tech['BB_Lower']) / df_tech['BB_Middle']

  df_tech.drop(columns=['BB_Upper', 'BB_Lower', 'BB_Middle'], inplace=True)

  df_tech.replace([np.inf, -np.inf], np.nan, inplace=True)

  df_tech.dropna(inplace=True)

  df_processed, ohe_col_names = preprocessing(df_tech, True)
  df_processed_updated = df_processed.drop(columns=['Stock'])
  input_dim = df_processed_updated.shape[1]

  shutil.rmtree('autoencoder_tuning/stock_data', ignore_errors=True)
# Instantiate the hypermodel
  hypermodel = AutoencoderHyperModel(input_dim=input_dim)

  # Initialize the Random Search tuner
  tuner = RandomSearch(
      hypermodel,
      objective='val_loss',
      max_trials=15,
      executions_per_trial=2,
      directory='autoencoder_tuning',
      project_name='stock_data'
  )

  tuner.search(df_processed_updated, df_processed_updated,
             epochs=50,
             batch_size=512,
             shuffle=True,
             validation_split=0.2)
  best_model = tuner.get_best_models(num_models=1)[0]

  encoder = Model(inputs=best_model.input, outputs=best_model.get_layer('dense_1').output)

  encoded_data = encoder.predict(df_processed_updated)

  kmeans = KMeans(n_clusters=3, random_state=42)
  kmeans.fit(encoded_data)

  # Get the cluster labels
  cluster_labels = kmeans.labels_

  df_tech_encoder_clustered = df_processed.copy()
  df_tech_encoder_clustered['Cluster'] = cluster_labels

  df_tech_encoder_clustered = df_tech_encoder_clustered.drop(columns=ohe_col_names)

  feature_summary = feature_summary_statistics(df_tech_encoder_clustered, 'Cluster', cols)

  cluster_risk_mapping = get_cluster_risk_mapping(feature_summary)

  df_tech_encoder_clustered['Risk_Label'] = df_tech_encoder_clustered['Cluster'].map(cluster_risk_mapping) #replace the df_tech_encoder_clustered by df_tech if neuro fuzzy output doesn't make sense

  df_tech_encoder_clustered.drop(columns='Cluster', inplace=True)

  return df_tech_encoder_clustered

In [ ]:
industry_to_df_mapping = {}
for industry in selected_industries:
  s = time.time()
  print(industry)
  df = pd.read_sql("select open, high, low, close, volume, dividends, stock_splits, stock from stock_prices where industry=?", conn, params=(industry, ))
  #print(len(df))
  df_out = perform_autoencoder_clustering(df)
  industry_to_df_mapping[industry] = df_out
  print(f"Time elapsed for {industry} is {time.time() - s}")

Trial 15 Complete [00h 00m 18s]
val_loss: 0.04045742563903332

Best val_loss So Far: 0.02220881450921297
Total elapsed time: 00h 04m 12s
329/329 [==============================] - 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-19-05c832a577a4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_stats_df = summary_stats_df.append({'Feature': col, 'Cluster': cluster_label,
<ipython-input-19-05c832a577a4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_stats_df = summary_stats_df.append({'Feature': col, 'Cluster': cluster_label,
<ipython-input-19-05c832a577a4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_stats_df = summary_stats_df.append({'Feature': col, 'Cluster': cluster_lab

Time elapsed for Medical Devices is 255.36590051651


In [ ]:
all_dfs = []
for industry, df in industry_to_df_mapping.items():
    # Add a new column with the industry name
    df['industry'] = industry
    # Append the modified DataFrame to the list
    all_dfs.append(df)

# Combine all the modified DataFrames into a single DataFrame
combined_df = pd.concat(all_dfs, ignore_index=True)

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(f'sqlite:///{output_path}')

In [ ]:
combined_df.to_sql('Clustered_output', con=engine, if_exists='replace', index=False)

120693

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Assume 'encoded_data_clusters' are your cluster labels from the autoencoder approach
# and 'mbkmeans_clusters' are your cluster labels from the Mini Batch KMeans approach

# Calculate metrics for Autoencoder-based clustering
silhouette_autoencoder = silhouette_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_encoder_clustered['Cluster'])
calinski_autoencoder = calinski_harabasz_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_encoder_clustered['Cluster'])
davies_autoencoder = davies_bouldin_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_encoder_clustered['Cluster'])

# Calculate metrics for Mini Batch KMeans clustering
silhouette_mbkmeans = silhouette_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_clustered['Cluster'])
calinski_mbkmeans = calinski_harabasz_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_clustered['Cluster'])
davies_mbkmeans = davies_bouldin_score(df_tech_encoder_clustered.drop(columns=['Cluster', 'Stock']), df_tech_clustered['Cluster'])

print("Autoencoder-based Clustering Evaluation:")
print("Silhouette Score:", silhouette_autoencoder)
print("Calinski-Harabasz Score:", calinski_autoencoder)
print("Davies-Bouldin Score:", davies_autoencoder)

print("\nMini Batch KMeans Clustering Evaluation:")
print("Silhouette Score:", silhouette_mbkmeans)
print("Calinski-Harabasz Score:", calinski_mbkmeans)
print("Davies-Bouldin Score:", davies_mbkmeans)

Autoencoder-based Clustering Evaluation:
Silhouette Score: 0.8261076795556598
Calinski-Harabasz Score: 46039.63515831459
Davies-Bouldin Score: 0.7788329814067539

Mini Batch KMeans Clustering Evaluation:
Silhouette Score: 0.5811297797045788
Calinski-Harabasz Score: 36826.133890233396
Davies-Bouldin Score: 0.7497034640186309
